In [1]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
import scqubits as scq
import pandas as pd

from qiskit_metal.analyses.hamiltonian.transmon_charge_basis import Hcpb
from qiskit_metal.analyses.hamiltonian.transmon_CPB_analytic import Hcpb_analytic
%matplotlib inline


In [22]:
# Ec is set to 250Mhz and a Ej/Ec ratio of 70 is initialised
E_C_main = 250
ratio_main = 70
E_J_main = ratio_main * E_C_main

# Pre-Pulse Hamiltonian
H_norm_i = Hcpb(nlevels=2, Ej=E_J_main, Ec=E_C_main, ng=0.5)
# Pulsed Hamiltonian
H_norm_p = Hcpb(nlevels=2, Ej=E_J_main*0.01, Ec=E_C_main, ng=0.5)


In [31]:
def plot_eigenenergies_offset_charge(ng_vec, E_J, E_C, max_level, ax):
    E_dict = {}
    H_norm = Hcpb(nlevels=2, Ej=E_J, Ec=E_C, ng=0.5)
    norm = H_norm.fij(0,1)


    for m in range(0, max_level+1):
        E_dict[f"E{m}"] = []

 
    for i in ng_vec:
        H = Hcpb(nlevels=max_level+1, Ej=E_J, Ec=E_C, ng=i)

        for m in range(0, max_level+1):
            E_dict[f"E{m}"].append(H.evalue_k(m)/norm)
    
    floor = min(E_dict["E0"])

    for m in range(0, max_level+1):
        ax.plot(ng_vec, E_dict[f"E{m}"])
    ax.set_xlabel("ng")
    ax.set_ylabel("Em/E01")
    ax.set_title(f"Ej:{E_J}, Ec:{E_C}, ratio:{E_J/E_C}")

def plot_charge_dispersion(ratio_vec, E_c, max_level, ax):
    ep_dict={}
    for m in range(0, max_level+1):
        ep_dict[f"ep{m}"] = []

    for i in ratio_vec:
        E_j = i*E_c
        H_zero = Hcpb(nlevels=15, Ej=i*E_c, Ec=E_c, ng=0.0)
        H_half = Hcpb(nlevels=15, Ej=i*E_c, Ec=E_c, ng=0.5)

        H_norm = Hcpb(nlevels=15, Ej=i*E_c, Ec=E_c, ng=0.5)
        norm = H_norm.fij(0,1)                         # normalization constant


        for m in range(0, max_level+1):
            ep_dict[f"ep{m}"].append(abs(H_half.evalue_k(m) - H_zero.evalue_k(m))/norm)
    
    for m in range(0, max_level+1):
        ax.plot(ratio_vec, ep_dict[f"ep{m}"], label=f"m={m}")
        
    ax.set_yscale("log")
    ax.set_xlabel("EJ/Ec")
    ax.set_ylabel("Epsilon/E01")
    ax.set_title(f"Ec:{E_c}")
    ax.legend(title="Energy Level", loc='upper right')

def plot_anharmonicity(ratio_vec, E_c, ax_n, ax_r):
    alpha = []
    alpha_r = []

    for i in ratio_vec:
        H_anharm = Hcpb(nlevels=15, Ej=i*E_c, Ec=E_c, ng=0.5)
        alpha.append(H_anharm.anharm())
        alpha_r.append(H_anharm.anharm()/H_anharm.fij(0,1))    


    ax_n.plot(ratio_vec,alpha)
    ax_n.set_xlabel("Ej/Ec")
    ax_n.set_ylabel("alpha")
    ax_r.plot(ratio_vec,alpha_r)
    ax_r.set_ylim(-0.2, 1.0)
    ax_r.set_xlabel("Ej/Ec") 
    ax_r.set_ylabel("alpha_r")
    ax_n.set_title(f"anharmonicity with Ec:{E_c}")
    ax_r.set_title(f"relative anharmonicity with Ec:{E_c}")

# Visualisation of the Characteristics of a Transmon

In [ ]:
## Constant ratio, varying EJ and Ec
fig, ax = plt.subplots(4, 2,  figsize=(15, 15))
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=100*70, E_C=100, max_level = 2, ax=ax[0][0])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=100*70*0.01, E_C=100, max_level = 2, ax=ax[0][1])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=250*70, E_C=250, max_level = 2, ax=ax[1][0])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=250*70*0.01, E_C=250, max_level = 2, ax=ax[1][1])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=750*70, E_C=750, max_level = 2, ax=ax[2][0])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=750*70*0.01, E_C=750, max_level = 2, ax=ax[2][1])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=1200*70, E_C=1200, max_level = 2, ax=ax[3][0])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=1200*70*0.01, E_C=1200, max_level = 2, ax=ax[3][1])
fig.tight_layout()
plt.savefig("images/const_ratio_vary_Ej_Ec_E-vs-ng.png")
plt.show()

In [ ]:
## Constant EJ, varying Ec
fig, ax = plt.subplots(4, 2,  figsize=(15, 15))
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=20000, E_C=100, max_level = 2, ax=ax[0][0])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=20000*0.01, E_C=100, max_level = 2, ax=ax[0][1])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=20000, E_C=250, max_level = 2, ax=ax[1][0])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=20000*0.01, E_C=250, max_level = 2, ax=ax[1][1])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=20000, E_C=750, max_level = 2, ax=ax[2][0])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=20000*0.01, E_C=750, max_level = 2, ax=ax[2][1])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=20000, E_C=1200, max_level = 2, ax=ax[3][0])
plot_eigenenergies_offset_charge(ng_vec=np.linspace(-2.0,2.0,101), E_J=20000*0.01, E_C=1200, max_level = 2, ax=ax[3][1])
fig.tight_layout()
plt.savefig("images/const_Ejmax_vary_Ec_E-vs-ng.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(3, 2,  figsize=(15, 15))

plot_charge_dispersion(ratio_vec=np.linspace(0.01, 100, 101), E_c=250, max_level=2, ax = ax[0][0])
plot_charge_dispersion(ratio_vec=np.linspace(0.01, 1, 101), E_c=250, max_level=2, ax = ax[0][1])
plot_charge_dispersion(ratio_vec=np.linspace(1, 100, 101), E_c=500, max_level=2, ax = ax[1][0])
plot_charge_dispersion(ratio_vec=np.linspace(0.01, 1, 101), E_c=500, max_level=2, ax = ax[1][1])
plot_charge_dispersion(ratio_vec=np.linspace(1, 100, 101), E_c=1000, max_level=2, ax = ax[2][0])
plot_charge_dispersion(ratio_vec=np.linspace(0.01, 1, 101), E_c=1000, max_level=2, ax = ax[2][1])
fig.tight_layout()
plt.savefig("images/charge_dispersion_varying_Ec.png")
plt.show()

In [ ]:
fig, ax = plt.subplots(3, 2,  figsize=(15, 15))
plot_anharmonicity(ratio_vec=np.linspace(0.1, 100, 101), E_c=250, ax_n = ax[0][0], ax_r=ax[0][1])
plot_anharmonicity(ratio_vec=np.linspace(0.1, 100, 101), E_c=500, ax_n = ax[1][0], ax_r=ax[1][1])
plot_anharmonicity(ratio_vec=np.linspace(0.1, 100, 101), E_c=1000,ax_n = ax[2][0], ax_r=ax[2][1])
fig.tight_layout()
plt.savefig("images/anharmonicity_varying_Ec.png")
plt.show()